<a href="https://colab.research.google.com/github/manrai/pythonsurvey/blob/master/Python_survey_and_NHANES.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# load Python modules
import pandas as pd
import io
import requests

In [0]:
# import NHANES 2013-14 BMX data
url = "https://raw.githubusercontent.com/manrai/pythonsurvey/master/bmx_8.csv"
s = requests.get(url).content
df = pd.read_csv(io.StringIO(s.decode('utf-8')))

In [0]:
# preview data
df.head()

,SEQN,SDMVPSU,SDMVSTRA,WTMEC2YR,RIDRETH1,RIDRETH3,RIDAGEYR,RIAGENDR,BMXWT,BMXRECUM,...,BMXARMC,BMXWAIST,BMXTHICR,BMXTRI,BMXSUB,BMXHTGROUP,BMXSAD1,BMXSAD2,BMXSAD3,BMXSAD4
0,73557.0,1,112,13481.042095,4,4,69,1,78.3,NaN,...,35.3,100.0,NaN,NaN,NaN,NaN,20.5,20.6,NaN,NaN
1,73558.0,1,108,24471.769625,3,3,54,1,89.5,NaN,...,34.7,107.6,NaN,NaN,NaN,NaN,24.2,24.5,NaN,NaN
2,73559.0,1,109,57193.285376,3,3,72,1,88.9,NaN,...,33.5,109.2,NaN,NaN,NaN,NaN,25.8,25.4,NaN,NaN
3,73560.0,2,109,55766.512438,3,3,9,1,32.2,NaN,...,21.0,61.0,NaN,NaN,NaN,NaN,14.8,15.0,NaN,NaN
4,73561.0,2,116,65541.871229,3,3,73,2,52.0,NaN,...,25.2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [0]:
# summarize data
df.describe()

,SEQN,SDMVPSU,SDMVSTRA,WTMEC2YR,RIDRETH1,RIDRETH3,RIDAGEYR,RIAGENDR,BMXWT,BMXRECUM,...,BMXARMC,BMXWAIST,BMXTHICR,BMXTRI,BMXSUB,BMXHTGROUP,BMXSAD1,BMXSAD2,BMXSAD3,BMXSAD4
count,10175.000000,10175.000000,10175.000000,10175.000000,10175.000000,10175.000000,10175.000000,10175.000000,9723.000000,1065.000000,...,9301.000000,8661.000000,0.0,0.0,0.0,0.0,7218.000000,7218.000000,358.000000,358.000000
mean,78644.000000,1.484128,110.926880,30585.180933,3.091892,3.289828,31.484128,1.508305,62.599054,81.631174,...,28.485765,87.272047,NaN,NaN,NaN,NaN,21.107869,21.089748,22.228492,22.203911
std,2937.413829,0.499773,4.260322,27941.005388,1.263305,1.613241,24.421651,0.499956,32.331616,14.219097,...,7.961971,22.542605,NaN,NaN,NaN,NaN,4.964898,4.972462,5.052650,5.056758
min,73557.000000,1.000000,104.000000,0.000000,1.000000,1.000000,0.000000,1.000000,3.100000,48.600000,...,10.400000,40.200000,NaN,NaN,NaN,NaN,10.000000,10.200000,10.900000,11.000000
25%,76100.500000,1.000000,107.000000,12561.598665,2.000000,2.000000,10.000000,1.000000,37.950000,69.700000,...,22.600000,71.200000,NaN,NaN,NaN,NaN,17.300000,17.300000,18.500000,18.500000
50%,78644.000000,1.000000,111.000000,20174.573544,3.000000,3.000000,26.000000,2.000000,65.300000,82.800000,...,29.300000,87.800000,NaN,NaN,NaN,NaN,20.700000,20.700000,22.400000,22.400000
75%,81187.500000,2.000000,115.000000,36748.217798,4.000000,4.000000,52.000000,2.000000,83.500000,93.200000,...,34.000000,102.800000,NaN,NaN,NaN,NaN,24.400000,24.375000,25.400000,25.300000
max,83731.000000,2.000000,118.000000,171395.264901,5.000000,7.000000,80.000000,2.000000,222.600000,115.100000,...,59.400000,177.900000,NaN,NaN,NaN,NaN,40.200000,40.200000,35.600000,35.600000


Here we define a basic Python version of svymean from library(survey):

In [0]:
def svymean(variable_col,weight_col,df):
  data = df[[variable_col,weight_col]]
  data = data.dropna()
  wts = data[weight_col].values.tolist()
  dat = data[variable_col].values.tolist()
  return sum(w*d for w,d in zip(wts,dat))/sum(wts)

Let's check agreement with the R-based estimates:

In [0]:
svymean("BMXBMI","WTMEC2YR",df)

27.07056164452085

In [0]:
svymean("BMXHEAD","WTMEC2YR",df)

41.66863220605581

In [0]:
svymean("BMXHT","WTMEC2YR",df)

161.89324039700136

Now we implement a Python version of svyvar and check agreement with R-based estimates for the same three variables:

In [0]:
def svyvar(variable_col,weight_col,df):
  data = df[[variable_col,weight_col]]
  data = data.dropna()
  wts = data[weight_col].values.tolist()
  dat = data[variable_col].values.tolist()
  n = df.shape[0]-df[weight_col].values.tolist().count(0)
  dat_bar = svymean(variable_col,weight_col,df)
  dat = [d-dat_bar for d in dat]
  return (n/(n-1))*sum(w*d*d for w,d in zip(wts,dat))/sum(wts)

In [0]:
svyvar("BMXBMI","WTMEC2YR",df)

60.49812487155186

In [0]:
svyvar("BMXHEAD","WTMEC2YR",df)

6.488911180588469

In [0]:
svyvar("BMXHT","WTMEC2YR",df)

384.5863899670593